In [1]:
# !pip install py_vollib

In [138]:
from volapi import VolCaseClient
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_vollib.black_scholes_merton.greeks.analytical import delta
import numpy as np
import pandas as pd
import re
import time
from IPython.display import clear_output

In [139]:
url='http://localhost:8000/v1/'
apikey={'X-API-Key':'QN1PO4LT'}
c2p_dict = {'RTM2C'+str(i):'RTM2P'+str(i) for i in range(45, 55)}
p2c_dict = {'RTM2P'+str(i):'RTM2C'+str(i) for i in range(45, 55)}
ava_ticker = ['RTM2C'+str(i) for i in range(45, 55)]+['RTM2P'+str(i) for i in range(45, 55)]
ava_ticker = ava_ticker+['RTM1C'+str(i) for i in range(45, 55)]+['RTM1P'+str(i) for i in range(45, 55)]



In [140]:
# Case Params
r = 0   # risk free rate
q = 0   # dividend rate
real_vol = 0.2 # real volatility at the beginning
rng_time = np.concatenate(
    (np.arange(75,150), np.arange(225, 300), np.arange(375, 450))) # efficient predicting zone

# Strat Params
alpha = 20 # order intensity
safe_rng = 0.01 # safe range width
rng_d = real_vol - safe_rng # downside safe zone
rng_u = real_vol + safe_rng # upside safe zone

# Connect and Wait for start
api = VolCaseClient(url, apikey)
api.case_status()

True

In [141]:
api.case_tick()

array([[291, 300,   2,   2]], dtype=int64)

In [142]:
options_limit = 800
stock_limit = 10000
options_limit

800

In [144]:
options_limit = 800
stock_limit = 8000
options_limit_per_order = 100

# while api.case_status() == False:
#     print('NOT STARTED YET')
#     time.sleep(2)

# Case begins
now_tick = api.case_tick()[0][0]
while True:
    try:
        while api.case_status() == True:
            try:
                # skip same tick
                if api.case_tick()[0][0] == now_tick:
                    continue
                else:
                    now_tick = api.case_tick()[0][0]


                ####### ========================+######
                ##### two-month options #######
                ####### ========================########
                t = (600 - now_tick - (api.case_tick()[0][2]-1)*300)/ 300 / 12 # time to maturity

                # get data
                pos = api.position()
                pos_ticker = list(pos[pos!=0].index)
                prc_bid = api.price(kind='bid')
                prc_ask = api.price(kind='ask')
                prc_last = api.price(kind='last')
                S_ask = api.price(ticker="RTM",kind='ask')
                S_bid = api.price(ticker="RTM",kind='bid')
                S_last = api.price(ticker="RTM",kind='last')
                call_list = [i for i in ava_ticker if '2C' in i]
                put_list = [i for i in ava_ticker if '2P' in i]

                try:
                    vol_news = api.news_kind("Announcement",is_last=True)["body"]
                    real_vol = int(re.findall(r"\d+", vol_news.values[0])[0]) / 100
                except:
                    pass

                # define safe zone piecewise
                if now_tick + (api.case_tick()[0][2]-1)*300 <= 540:
                    rng_d = real_vol - 0.01
                    rng_u = real_vol + 0.01
                else:
                    rng_d = real_vol - 0.02
                    rng_u = real_vol + 0.02

                range_news = api.news_kind("News",is_last=True)["body"]
                try:
                    rng_d = int(re.findall(r"\d+", range_news.values[0])[0]) / 100
                    rng_u = int(re.findall(r"\d+", range_news.values[0])[1]) / 100
                    if rng_d == rng_u:
                        rng_d += 0.01
                        rng_u -= 0.01
                except:
                    pass

                # computation -- implied vol
                vol_dict = {}
                nan_count=[]
                for call in call_list:
                    K, flag = int(call[-2:]), 'c'
                    try:
                        iv = implied_volatility(prc_last[call], S_last, K+0.02, t, r, flag)
                    except Exception:
                        iv = np.nan
                        nan_count.append(call)
        #                 print('unsolvable for stock', S_last, ' ', flag, ' option of', K, 'has price', prc_last[call])
                    vol_dict[call] = iv

                for put in put_list:
                    K, flag = int(put[-2:]), 'p'
                    try:
                        iv = implied_volatility(prc_last[put], S_last, K-0.02, t, r, flag)
                    except Exception:
                        iv = np.nan
                        nan_count.append(put)
        #                 print('unsolvable for stock', S_last, ' ', flag, ' option of', K, 'has price', prc_last[put])
                    vol_dict[put] = iv
        #         print('the following two-month options has nan iv:', nan_count)
                iv_s = pd.Series(vol_dict)

                # find at-the-money option and its iv
                at_money_opt = []

                for i in iv_s.index:
                    if str(int(round(S_last))) in i:
                        at_money_opt.append(i)

                now_iv = iv_s[at_money_opt].mean()

                print("rv =", real_vol,  "iv =", round(now_iv,2), "safe range", round(rng_d,2), "~", round(rng_u,2))

                ## if iv is beyong the upper bound
                if (now_iv > rng_u)&(abs(api.position().iloc[1:].sum())<options_limit):
                    amount = min(int((now_iv - rng_u)/0.1*alpha+1), options_limit_per_order)
                    for opt in at_money_opt:
                        api.market_sell(opt, amount)
                    exec_prc = prc_bid[at_money_opt].sum()
                    print("sell straddle at", round(exec_prc,2), "by", amount)

                elif (now_iv < rng_d)&(abs(api.position().iloc[1:].sum())<options_limit):
                    amount = min(int((rng_d - now_iv)/0.1*alpha+1), options_limit_per_order)
                    for opt in at_money_opt:
                        api.market_buy(opt, amount)
                    exec_prc = prc_ask[at_money_opt].sum()
                    print("buy straddle at", round(exec_prc,2), "by", amount)

        #             print('start guessing volatility skew')

                    in_money_opt = []
                    for i in iv_s.index:
                        if ((int(round(S_last))-2 <= int(i[-2:]))&(int(round(S_last)) > int(i[-2:]))) & ('C' in i):
                            in_money_opt.append(i)

                    for opt in in_money_opt:
                        opts = [opt]+[opt.replace('C','P')]
                        now_iv = iv_s[opts].mean()
                        if now_iv < rng_d:
                            amount = min(int((rng_d - now_iv)/0.1*alpha), options_limit_per_order)
                            api.market_buy(opt, amount)
                            api.market_buy(opt.replace('C','P'), amount)
                            exec_prc = prc_ask[opts].sum()
                            print("buy straddle", opt, " at", round(exec_prc,2), "by", amount)
                        elif (now_iv >= rng_d) & (opt in pos_ticker):
                            for ticker in opts:
                                api.close_pos(ticker)
                                print("close", ticker, "by", amount)


                elif (now_iv<=rng_u)&(now_iv>=rng_d):
                    for ticker in iv_s.index.tolist():
                        if ticker in pos_ticker:
                            amount = api.close_pos(ticker)
                            print("close", ticker, "by", amount)
        #             continue

                ####### ========================+######
                ##### one-month options #######
                ####### ========================########
                print(now_tick)

                t1 = (300 - now_tick - (api.case_tick()[0][2]-1)*300)/ 300 / 12 # time to maturity
                if t1>0:

                    # get data
                    pos = api.position()
                    pos_ticker = list(pos[pos!=0].index)
                    prc_bid = api.price(kind='bid')
                    prc_ask = api.price(kind='ask')
                    prc_last = api.price(kind='last')
                    S_ask = api.price(ticker="RTM",kind='ask')
                    S_bid = api.price(ticker="RTM",kind='bid')
                    S_last = api.price(ticker="RTM",kind='last')
                    call_list = [i for i in ava_ticker if '1C' in i]
                    put_list = [i for i in ava_ticker if '1P' in i]

                    # computation -- implied vol
                    vol_dict = {}
                    nan_count=[]
                    for call in call_list:
                        K, flag = int(call[-2:]), 'c'
                        try:
                            iv = implied_volatility(prc_last[call], S_last, K+0.02, t1, r, flag)
                        except Exception:
                            iv = np.nan
                            nan_count.append(call)
        #                     print('unsolvable for stock', S_last, ' ', flag, ' option of', K, 'has price', prc_last[call])
                        vol_dict[call] = iv

                    for put in put_list:
                        K, flag = int(put[-2:]), 'p'
                        try:
                            iv = implied_volatility(prc_last[put], S_last, K-0.02, t1, r, flag)
                        except Exception:
                            iv = np.nan
                            nan_count.append(put)
        #                     print('unsolvable for stock', S_last, ' ', flag, ' option of', K, 'has price', prc_last[put])
                        vol_dict[put] = iv
        #             print('the following one-month options has nan iv:', nan_count)
                    iv_ss = pd.Series(vol_dict)

                    # find at-the-money option and its iv
                    at_money_opt = []

                    for i in iv_ss.index:
                        if (str(int(round(S_last))) in i):
                            at_money_opt.append(i)

                    now_iv = iv_ss[at_money_opt].mean()

                    print("rv =", real_vol,  "iv =", round(now_iv,2), "safe range", round(rng_d,2), "~", round(rng_u,2))

                    ## if iv is beyong the upper bound
                    if (now_iv > rng_u)&(abs(api.position().iloc[1:].sum())<options_limit):
                        amount = min(int((now_iv - rng_u)/0.1*alpha+1), options_limit_per_order)
                        for opt in at_money_opt:
                            api.market_sell(opt, amount)
                        exec_prc = prc_bid[at_money_opt].sum()
                        print("sell straddle at", round(exec_prc,2), "by", amount)

                    elif (now_iv < rng_d)&(abs(api.position().iloc[1:].sum())<options_limit):
                        amount = min(int((rng_d - now_iv)/0.1*alpha+1), options_limit_per_order)
                        for opt in at_money_opt:
                            api.market_buy(opt, amount)
                        exec_prc = prc_ask[at_money_opt].sum()
                        print("buy straddle at", round(exec_prc,2), "by", amount)

        #                 print('start guessing volatility skew')

                        in_money_opt = []
                        for i in iv_ss.index:
                            if ((int(round(S_last))-2 <= int(i[-2:]))&(int(round(S_last)) > int(i[-2:]))) & ('C' in i):
                                in_money_opt.append(i)

                        for opt in in_money_opt:
                            opts = [opt]+[opt.replace('C','P')]
                            now_iv = iv_ss[opts].mean()
                            if now_iv < rng_d:
                                amount = min(int((rng_d - now_iv)/0.1*alpha), options_limit_per_order)
                                api.market_buy(opt, amount)
                                api.market_buy(opt.replace('C','P'), amount)
                                exec_prc = prc_ask[opts].sum()
                                print("buy straddle", opt, " at", round(exec_prc,2), "by", amount)
                            elif (now_iv >= rng_d) & (opt in pos_ticker):
                                for ticker in opts:
                                    api.close_pos(ticker)
                                    print("close", ticker, "by", amount)

                    elif (now_iv<=rng_u)&(now_iv>=rng_d):
                        for ticker in iv_ss.index.tolist():
                            if ticker in pos_ticker:
                                amount = api.close_pos(ticker)
                                print("close", ticker, "by", amount)
            #             continue


                # hedge
                # delta hedge
                try:
                    iv_s = iv_s.append(iv_ss)
                except:
                    pass

                pos = api.position()
                pos_ticker = list(pos[pos!=0].index)
                sum_delta = 0
                tmp  = 0
                for opt_ticker in pos_ticker:
                    if 'C' in opt_ticker:
                        flag = 'c'
                    elif 'P' in opt_ticker:
                        flag = 'p'
                    else:
                        sum_delta += pos[opt_ticker] * 1
                        tmp += pos[opt_ticker] * 1
                        continue
                    K = int(re.findall(r'\d+', opt_ticker)[0])
                    if opt_ticker in call_list:
                        sigma = iv_s[opt_ticker]
                    elif opt_ticker in put_list:
                        sigma = iv_s[opt_ticker]
        #             sigma = (rng_d+rng_u)/2

                    if ('1C' in opt_ticker)|('1P' in opt_ticker):
                        tt = t1
                    else:
                        tt = t
                    opt_delta = delta(flag, S_last, K, tt, r, sigma, q)
                    sum_delta +=  pos[opt_ticker] * 100 * opt_delta

                print('delta is:', sum_delta, ',delta from ETF is', tmp)

                if sum_delta==tmp:
                    if sum_delta>0:
                        while sum_delta > stock_limit:
                            api.market_sell("RTM", stock_limit)
                            sum_delta -= stock_limit
                        api.market_sell("RTM", sum_delta)
                    elif sum_delta<0:
                        while sum_delta < -stock_limit:
                            api.market_buy("RTM", stock_limit)
                            sum_delta += stock_limit
                        api.market_buy("RTM", -sum_delta)
                else:
                    if sum_delta > 0:
                        while sum_delta > stock_limit:
                            api.market_sell("RTM", stock_limit)
                            sum_delta -= stock_limit
                        api.market_sell("RTM", sum_delta)
                    elif sum_delta < 0:
                        while sum_delta < -stock_limit:
                            api.market_buy("RTM", stock_limit)
                            sum_delta += stock_limit
                        api.market_buy("RTM", -sum_delta)

                    # time.sleep(1)

            except:
                time.sleep(3)
        
        clear_output(wait=True)
        print('Not Start Yet')
        time.sleep(3)
        
    except:
        time.sleep(3)

Not Start Yet


KeyboardInterrupt: 

In [120]:
for ticker in pos_ticker:
    amount = api.close_pos(ticker)
    print("close", ticker, "by", amount)

close RTM by 100.0
close RTM1C49 by -1.0
close RTM1P49 by -1.0
close RTM1C50 by -1.0
close RTM1P50 by -1.0


In [63]:
api.position().iloc[1:].sum()

-212.0

In [90]:
iv_s

RTM2C45    0.215073
RTM2C46    0.212038
RTM2C47    0.210141
RTM2C48    0.210363
RTM2C49    0.208272
RTM2C50    0.208310
RTM2C51    0.209824
RTM2C52    0.210867
RTM2C53    0.203594
RTM2C54    0.233221
RTM2P45    0.206165
RTM2P46    0.206989
RTM2P47    0.207216
RTM2P48    0.208823
RTM2P49    0.207878
RTM2P50    0.209167
RTM2P51    0.212411
RTM2P52    0.216347
RTM2P53    0.215962
RTM2P54    0.232058
dtype: float64

In [32]:
S_last

53.71

In [ ]:
iv_s.index

In [80]:
str(int(round(S_last)))

'47'